In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from snmfem.estimator.nmf import NMF
from snmfem.measures import KLdiv_loss
from snmfem.toy import create_toy_problem

In [ ]:
l = 25
k = 3
p = 100
c = 10
n_poisson=200
G, P, A, X, Xdot = create_toy_problem(l, k, p, c, n_poisson)
G.shape

In [ ]:
est = NMF(G=G, n_components=k, debug=True, max_iter=2000,  force_simplex=False)

In [ ]:
Ps, As = est.fit_transform(X, eval_print=20)

In [ ]:
est = NMF(G=G, n_components=k, debug=True, max_iter=2000, force_simplex=False)
Pss, Ass = est.fit_transform(Xdot, eval_print=200)

In [ ]:
vmin = 0
vmax = 0.15

plt.subplot(2,1,1)
plt.imshow(X, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.subplot(2,1,2)
plt.imshow(Xdot, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.show()

In [ ]:
np.max(Xdot)